In [1]:
%load_ext autoreload
%autoreload 2

from src import *
import tensorflow as tf
import numpy as np

In [2]:
batch_size = 1024
epochs = 10
image_feat_path = "./data/test_data/flickr30k/flickr30k_vgg_image_feat_test.mat"
sent_feat_path = "./data/test_data/flickr30k/flickr30k_text_feat_test.mat"
shuffle_step=5

In [3]:
data_loader = DatasetLoader(image_feat_path, sent_feat_path)

Loading image features from ./data/test_data/flickr30k/flickr30k_vgg_image_feat_test.mat
Loaded image feature shape: (1000, 4096)
Loading sentence features from ./data/test_data/flickr30k/flickr30k_text_feat_test.mat
Loaded sentence feature shape: (5000, 6000)


In [4]:



model = embedding_model(data_loader.im_feats.shape[1:], data_loader.sent_feats.shape[1:])
loss = EmbeddingLoss(1, batch_size=batch_size)
# model.summary()

In [5]:



sent_sample_size = 5 # how many sentences per image

total_loss, batch_loss = [], []
optimizer = tf.keras.optimizers.Adam()

In [6]:
def _get_data(data_loader):
    img_feats = data_loader.im_feats
    sent_feats = data_loader.sent_feats  

    def gen():
        for ind, img in enumerate(img_feats):
            for i in range(0,5):
                yield img, sent_feats[ind*5+i]
    return gen

data = tf.data.Dataset.from_generator(_get_data(data_loader), (tf.float32, tf.float32)).shuffle(1000).batch(batch_size)

In [22]:
optimizer = tf.keras.optimizers.Adam()
labels = np.repeat(np.eye(batch_size, dtype=bool), 1, axis=0)
for epoch in range(10):
    for im_feats, sent_feats in data:
        _batch_loss = tf.constant(0, dtype=tf.float32)
        with tf.GradientTape() as tape:
            [im_emb, sent_emb] = model([im_feats, sent_feats])
            
            _batch_loss += loss(im_emb, sent_emb, labels)

        trainable_variables = model.trainable_variables

        gradients = tape.gradient(_batch_loss, trainable_variables)
        optimizer.apply_gradients(zip(gradients, trainable_variables))

        total_loss.append(_batch_loss / int(im_feats.shape[1]))

InvalidArgumentError: Input to reshape is a tensor with 1024 values, but the requested shape has 1000 [Op:Reshape]

### currently the data looks like 

| sample size | img_feats | sent_feats |
|-------------|-------------|-------------|
| 1           | img1      | [s1, s2s3, s4, s5] | 
| 2           | img2      | [s1, s2, s3, s4, s5] | 

we need it to be 

|sample size | img_feats | sent_feats|
|-------------|-------------|-------------|
|1           | img1      | s1|
|1           | img1      | s2|
|1           | img1      | s3|
|1           | img1      | s4|
|1           | img1      | s5|
|1           | img2      | s1|
|2           | img2      | s2 and so on

In [8]:
im_feats.shape

(1024, 4096)

In [9]:
sent_feats.shape

(5120, 6000)

In [42]:
feat_list = []

for ind, img in enumerate(im_feats):
    for i in range(0,5):
        feat_list.append([img, sent_feats[ind*5+i]])
    

In [11]:
print(np.array(img_feat_list).shape)
print(np.array(sent_feat_list).shape)

(5120, 4096)
(5120, 6000)
